In [36]:
import time
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv

load_dotenv()

SCREEN_WIDTH = 1470
SCREEN_HEIGHT = 956
HEADLESS = True
MAX_TIME_TO_WAIT = 20

# Cria o driver e ajusta para que a tela do navegador fique do lado esquerdo
options = Options()
options.add_argument('--disable-gpu')  # Desabilita a aceleração de GPU (opcional)
if HEADLESS: options.add_argument('--headless')  # Ativa o modo headless
driver = webdriver.Chrome(options=options)
if not HEADLESS:
    driver.set_window_position(0, 0)
    driver.set_window_size(SCREEN_WIDTH // 2, SCREEN_HEIGHT)  # Para ficar do lado esquerdo
driver.get('https://ufabcnext.com/')

# Localiza botão do Facebook e clica
facebook_button = WebDriverWait(driver, MAX_TIME_TO_WAIT).until(
    EC.element_to_be_clickable((By.CLASS_NAME, 'facebook-button'))
)
facebook_button.click()

# Escreve o e-mail e RA
input_email = WebDriverWait(driver, MAX_TIME_TO_WAIT).until(
    EC.presence_of_element_located((By.ID, 'input-0'))
)
input_email.send_keys(os.environ['EMAIL'])
input_ra = driver.find_element(by=By.ID, value='input-2')  # Não necessário usar o WebDriverWait -> Se já encontrou um, o outro tem que estar lá também 
input_ra.send_keys(os.environ['RA'])
submit_button = driver.find_element(by=By.CSS_SELECTOR, value='button[type="submit"]')
submit_button.click()

# Procura por uma matéria em específico
subjects_input = WebDriverWait(driver, MAX_TIME_TO_WAIT).until(
    EC.presence_of_element_located((By.ID, 'input-20'))
)
subjects_input.send_keys('bases matem')
option = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'v-list-item')][contains(., 'bases matemáticas')]"))
)
option.click()

# Puxa os valores e imprime na tela
grades_percentage = WebDriverWait(driver, MAX_TIME_TO_WAIT).until(
    EC.visibility_of_all_elements_located((By.CLASS_NAME, 'highcharts-text-outline'))
)
for gp in grades_percentage:
    gp_text = gp.text
    grade, percentual = gp_text.split(':')[:2]  # Pega o 0 e 1
    percentual = percentual.split('%')[0].strip()

    print(f'{grade}: {percentual}%')
    
time.sleep(5)
driver.quit()

A: 8.8%
B: 15.4%
C: 25.8%
D: 15.5%
F: 29.0%
O: 5.5%
